In [1]:
%%time

from config.config_lv_grids_osm import get_config_osm
from config.db_conn_local import create_session_osm 
from config.classes_db_conn.osm_load_classes import Buildings_with_Amenities, Building_wo_Amenity, Amenities_ni_Buildings, Way

from grid.lv_grid.parameterization import parameterize_by_load_profiles, parameterize_by_load_profiles_IT_DEPRECATED

from grid.lv_grid.routing import build_graph_from_ways
from shapely import wkt
from sqlalchemy import func

import pandas as pd
from sqlalchemy import create_engine
from geoalchemy2.shape import to_shape
from config.config_lv_grids_osm import get_peak_loads, get_load_profile_categories, get_config_osm 


Wall time: 13.4 s


In [2]:
session_osm = create_session_osm()


# polies from small to big
# ca 25 buildings
#polygon = wkt.loads(u'POLYGON ((9.444757321553571 47.6640820372137, 9.44469147049122 47.6642370363376, 9.442746638836271 47.6675223477341, 9.44273323863089 47.6675486765942, 9.44272335456083 47.6675846924473, 9.44272337322715 47.6675856076278, 9.44272365679015 47.667586502615, 9.442724194246059 47.6675873426787, 9.442724964738851 47.6675880952201, 9.442725938369239 47.6675887310364, 9.442771524985689 47.6676085195463, 9.442809078141879 47.6676199425426, 9.443599054706221 47.6678292167847, 9.44364298889086 47.6678391758214, 9.44369319196211 47.6678454078942, 9.443694609235431 47.6678454056071, 9.44369600193242 47.6678452280573, 9.443697323077419 47.6678448812336, 9.443698528108349 47.6678443768344, 9.443744598701199 47.6678160313241, 9.443795360184319 47.6677784822092, 9.443991191770809 47.6676599114601, 9.44421793991903 47.667570002951, 9.44446657647225 47.667512336406, 9.44472720190474 47.6674892078198, 9.444989439424219 47.6675015380556, 9.445052589744799 47.6675088406224, 9.445117064489731 47.6675174206873, 9.445183601261711 47.6675274468952, 9.44523747329856 47.6675300193568, 9.445238809309229 47.6675299032106, 9.4452400852424 47.6675296113077, 9.445241251079061 47.667529155091, 9.445242261116221 47.6675285524453, 9.445243075758549 47.6675278269953, 9.44524366307046 47.6675270071801, 9.445257172339209 47.6674916444378, 9.44526138400154 47.6674643786881, 9.44526635876986 47.6674027181877, 9.44527842375639 47.6673216144847, 9.44528702520314 47.6672820624034, 9.445343690252869 47.6671237851091, 9.44544200701166 47.6669751340207, 9.445578690330031 47.6668410756375, 9.445606929899901 47.666818179473, 9.445637520256669 47.6667867600046, 9.445694232828229 47.6667392820943, 9.445718033000571 47.6667110812337, 9.445816550854619 47.6665782840333, 9.445885789521761 47.6664961432372, 9.44592142419992 47.6664586450851, 9.44609642847751 47.6663115101204, 9.446313359526091 47.6661926345661, 9.44656247099808 47.6661073591621, 9.4468325709589 47.6660595150674, 9.447111524661491 47.6660512517601, 9.44717730910928 47.6660540322315, 9.447316189895529 47.6660649297759, 9.44753610831893 47.6660902456686, 9.44758112940486 47.6660938580278, 9.44767071891042 47.6660918313842, 9.447684852061499 47.6660900296329, 9.44774691292624 47.6660795923001, 9.4479881800759 47.6660547725018, 9.448232085468209 47.6660604809193, 9.448470258112661 47.6660965216365, 9.448560914386359 47.6661163562617, 9.44857404167368 47.6661182063231, 9.448691583060739 47.6661430580836, 9.44873711713312 47.6661476818212, 9.44936429835351 47.6661892570172, 9.44940607834077 47.66619064719, 9.44946122599818 47.6661870380657, 9.449463401923399 47.6661858503929, 9.449498075077059 47.6661578721339, 9.449520006942709 47.6661337837734, 9.45042894326825 47.6650175089795, 9.45044714891934 47.6649921250378, 9.450463499380801 47.6649559172176, 9.450463614247271 47.6649550512483, 9.450463463599689 47.664954187754, 9.450463053866351 47.6649533635807, 9.45046240253088 47.6649526138965, 9.450461537386341 47.6649519706911, 9.450460495349169 47.6649514614106, 9.45040935260457 47.6649360626301, 9.45036907619812 47.664928709451, 9.447377834868041 47.6644933952575, 9.447343269935271 47.6644880343118, 9.44730847614896 47.6644823034133, 9.44727413690042 47.6644763151236, 9.444928887305879 47.6640444576946, 9.444886220360861 47.6640381906818, 9.444829205115131 47.6640359557828, 9.4448279422305 47.6640360849357, 9.44482674602855 47.6640363876611, 9.44482566804831 47.6640368509158, 9.444757321553571 47.6640820372137))')
# ca 150 buildings 
#polygon = wkt.loads(u'POLYGON((9.48065814797112 47.702048792915,9.48067670106068 47.7021395014346,9.48066347764271 47.7023084812701,9.48060322151733 47.7024727172123,9.48049810585165 47.7026262848256,9.48035192210726 47.7027636444594,9.48016994331629 47.7028798410926,9.4799587338921 47.7029706830933,9.47972591283223 47.7030328934435,9.47947987885587 47.7030642279683,9.47934922845884 47.7030722981368,9.47934792440673 47.7030725587507,9.47934672068944 47.7030729863116,9.47934566356528 47.7030735643887,9.47934479365881 47.7030742707668,9.47934414440016 47.7030750783002,9.47934374073991 47.7030759559558,9.479336776973 47.7031206087375,9.47932811383377 47.7039767787497,9.47930419238297 47.7041397502748,9.47923653630674 47.7042970032425,9.47912743128483 47.7044432241517,9.47898056362106 47.704573472226,9.4788008957201 47.7046833463729,9.47859449842742 47.7047691339106,9.47836834589566 47.7048279360353,9.47813007990952 47.7048577657854,9.47799942474244 47.7048658343318,9.47799812063796 47.7048660949293,9.47799691686738 47.704866522475,9.47799585969107 47.7048671005385,9.4779949897356 47.7048678069052,9.47799434043296 47.7048686144299,9.47799393673543 47.7048694920797,9.47798697154557 47.7049141447356,9.4779787295344 47.7057265448722,9.47798478818284 47.7057712596147,9.47798517404136 47.7057721410293,9.47798580690837 47.7057729546676,9.47798666246323 47.7057736692622,9.47798770782735 47.7057742573515,9.47798890282809 47.7057746963355,9.4779902015421 47.7057749693445,9.47805627420604 47.7057796855643,9.47925836179254 47.7057854103959,9.47932452365425 47.7057813239307,9.47939684134063 47.7057668720675,9.47947074051022 47.7057461669742,9.47970464719749 47.7057122633707,9.4799437768382 47.7057074870088,9.48018025507286 47.7057319951709,9.48034744636498 47.7057711862927,9.48037689470352 47.7057757983759,9.48037825014002 47.7057758265235,9.48043195925295 47.7057697338835,9.48047172866012 47.7057610034903,9.48133621703282 47.7055388618592,9.48135885400651 47.7055327249778,9.48138130051303 47.705526317869,9.48140359308013 47.7055196301061,9.48184265548561 47.7053814134536,9.48188497300641 47.7053668406817,9.48192634797981 47.7053513243399,9.48196703670106 47.705334768313,9.48261598271012 47.7050491257199,9.48284333280551 47.704969152505,9.48308929237145 47.7049204267937,9.48334456355668 47.7049047905248,9.48459384072616 47.704910679556,9.48466000103303 47.7049065898632,9.48466130510465 47.7049063291863,9.48466250882066 47.7049059015673,9.48466356592275 47.7049053234393,9.48466443578721 47.7049046170195,9.48466508498556 47.7049038094553,9.48466548856953 47.7049029317807,9.48467244796152 47.7048582786949,9.48467252083228 47.7048510026062,9.48470144355156 47.7046713355684,9.48478341991812 47.7044993167496,9.48491507631957 47.7043420242559,9.48509099507982 47.7042059301838,9.48530393743215 47.7040966343336,9.48554514139163 47.7040186338194,9.48580468226796 47.7039751380528,9.48607188098854 47.7039679367057,9.48633574343929 47.703997326084,9.48658541275859 47.704062096937,9.48665996453331 47.7040875703233,9.48666125058864 47.7040878085908,9.4867167774489 47.7040903270379,9.48675968026844 47.7040879027794,9.48836847986512 47.7039418050495,9.4884117301121 47.7039363032622,9.48846597903209 47.7039231915254,9.48846705282667 47.7039227432537,9.48846796485505 47.7039221536213,9.48846867510225 47.7039214484983,9.48846915240659 47.7039206588216,9.48847935080848 47.7038823344878,9.48848025539496 47.703852595877,9.48842973702086 47.7027828505263,9.48844378123493 47.7026216774512,9.48850063100891 47.7024648693601,9.48859841913968 47.7023175754942,9.48865532790434 47.7022617469993,9.48868669396787 47.702222722459,9.48868709746194 47.7022218447685,9.48869405298831 47.702177191366,9.48870212495492 47.7013647888932,9.48869605749258 47.7013200746416,9.48869567148364 47.7013191932628,9.48869503850179 47.7013183796845,9.48869418287212 47.7013176651721,9.48869313747609 47.7013170771837,9.48869194248755 47.7013166383156,9.4886906438294 47.7013163654331,9.48862457593146 47.7013116556256,9.48742259199815 47.7013060337188,9.4873564365445 47.7013101250925,9.48735513257782 47.701310385802,9.48735392896852 47.7013108134515,9.48735287197076 47.7013113916065,9.48735200220425 47.701312098049,9.48735135309372 47.7013129056306,9.48733547721792 47.7013474377041,9.48729880272183 47.7014642066402,9.48718750471712 47.7016336934443,9.48702670182373 47.7017839298192,9.48682336012544 47.7019084070581,9.48658628881307 47.7020017323772,9.48632575854415 47.7020598625765,9.48623025600094 47.7020739303646,9.48617046017902 47.7020907513363,9.48604806357768 47.7021335412969,9.48580331253464 47.7021817585738,9.48554939661524 47.70219722886,9.48475058108646 47.702193471732,9.48468442427628 47.7021975614879,9.4846831202735 47.7021978221655,9.48468191662114 47.7021982497853,9.48468085957504 47.7021988279143,9.48461736729804 47.7022503929486,9.48443111462172 47.7023742811523,9.48421235274966 47.7024708953687,9.48396968035719 47.7025364380003,9.48371263609498 47.702568332763,9.48345132361183 47.7025653259663,9.48319601435324 47.7025275357989,9.48295674375924 47.7024564476832,9.48274291674611 47.7023548558783,9.48256293798974 47.7022267536309,9.48242388155127 47.7020771761945,9.48233121283449 47.7019120028899,9.48225890353158 47.701730597702,9.48224568495493 47.701703164647,9.4822201219771 47.7016691405087,9.48221931544813 47.7016684616587,9.48221831995788 47.7016679091309,9.48221717790197 47.701667506456,9.48221593791797 47.7016672707831,9.48221465281409 47.7016672121489,9.48215874690368 47.7016725113534,9.48211790122247 47.7016808820075,9.48167200443362 47.7017895566957,9.4816344148702 47.7018000918094,9.48158928266617 47.7018181334203,9.48157294080893 47.7018278627516,9.48136857897516 47.7019277897901,9.48113978789522 47.7019995726057,9.48089494607431 47.70204058243,9.48065814797112 47.702048792915))')
# ca 4000
polygon = wkt.loads(u'POLYGON ((9.1641 48.8885, 9.1641 48.9017, 9.1935 48.9017, 9.1935 48.8885, 9.1641 48.8885))')


In [3]:
%%time

### Load bays
ways = session_osm.query(Way).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Way.geometry)) 


graph = build_graph_from_ways(ways, polygon, False, False)


### Load buildings_w_a
buildings_w_a = session_osm.query(Buildings_with_Amenities).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Buildings_with_Amenities.geometry_amenity)) 

### Load buildings_wo_a
buildings_wo_a = session_osm.query(Building_wo_Amenity).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Building_wo_Amenity.geometry)) 

### Load buildings_wo_a
amenities_ni_Buildings = session_osm.query(Amenities_ni_Buildings).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Amenities_ni_Buildings.geometry)) 


# parameterization and merge to buildings_df
buildings_w_loads_deprecated = parameterize_by_load_profiles_IT_DEPRECATED(buildings_w_a, buildings_wo_a, amenities_ni_Buildings)

bicycle_parking
parking_entrance
Wall time: 27.1 s


In [4]:
# sqlachemy local osm data
DB = 'postgresql'
DB_USER = 'postgres'
DB_PW   = 'labor'
DB_Name = 'sandbox_bw' 
engine_osm = create_engine(DB + '://' + DB_USER + ': ' + DB_PW + '@localhost:5432/' + DB_Name, echo=False)
session_osm = create_session_osm()


In [5]:
%%time


### Load bays
ways = session_osm.query(Way).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Way.geometry)) 


graph = build_graph_from_ways(ways, polygon, False, False)

### Load buildings_w_a
buildings_w_a = session_osm.query(Buildings_with_Amenities).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Buildings_with_Amenities.geometry_amenity)) 

### Load buildings_wo_a
buildings_wo_a = session_osm.query(Building_wo_Amenity).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Building_wo_Amenity.geometry)) 

### Load buildings_wo_a
amenities_ni_Buildings = session_osm.query(Amenities_ni_Buildings).filter(func.st_intersects(func.ST_GeomFromText(polygon.wkt, get_config_osm('srid')), Amenities_ni_Buildings.geometry)) 




# start with amenities due to they have no shape, only a point, 
# thus, they are handled a ltiitle bit different
amenities_ni_Buildings_sql_df = pd.read_sql(
    amenities_ni_Buildings.statement,
    #con=session_osm.bind both ways are working. select the easier/ more appropriate one
    con=engine_osm
)

buildings_w_a_sql_df = pd.read_sql(
    buildings_w_a.statement,
    con=engine_osm
)

buildings_wo_a_sql_df = pd.read_sql(
    buildings_wo_a.statement,
    con=engine_osm
)


buildings_w_loads_df = parameterize_by_load_profiles(amenities_ni_Buildings_sql_df, buildings_w_a_sql_df, buildings_wo_a_sql_df)


bicycle_parking
parking_entrance
Wall time: 2.96 s


In [6]:
print(len(buildings_w_loads_df))
buildings_w_loads_df.capacity.sum()

3786


90051.73943655926

In [7]:
print(len(buildings_w_loads_deprecated))
buildings_w_loads_deprecated.capacity.sum()

3786


76645.38841546506

In [11]:
buildings_w_loads_df.sort_values('capacity', ascending=False)

,category,area,geometry,raccordement_building,name,number_households,n_amenities_inside,osm_id_building,geometry_amenity,x,y,capacity
208859161,industrial,37356.600,"POLYGON ((9.165388699999999 48.89100459971936,...",POINT (9.166169849276244 48.89182932279353),None,0.666667,1.0,NaN,NaN,9.16617,48.891829,6145.160700
208885938,industrial,32156.800,"POLYGON ((9.170302 48.8932507997188, 9.1706538...",POINT (9.171253250360374 48.89371427246718),STIHL Werk 5,1.489362,1.0,NaN,NaN,9.171253,48.893714,5289.793600
132279038,industrial,15819.300,"POLYGON ((9.166468 48.8940049997186, 9.1664683...",POINT (9.167003435859485 48.89340192480272),Lotter,1.000000,1.0,NaN,NaN,9.167003,48.893402,2602.274850
132279034,industrial,12396.400,"POLYGON ((9.165444600000001 48.89376189971868,...",POINT (9.16596954408331 48.89362979538151),None,1.823529,1.0,NaN,NaN,9.16597,48.89363,2039.207800
139755343,industrial,11885.900,"POLYGON ((9.1784263 48.88814799972007, 9.17889...",POINT (9.178967405712543 48.88844672032332),None,0.787879,1.0,NaN,NaN,9.178967,48.888447,1955.230550
...,...,...,...,...,...,...,...,...,...,...,...,...
567262993,place_of_worship,1079.640,"POLYGON ((9.192986599999999 48.89974089971719,...",POINT (9.19312181385466 48.89964325600684),Hrvatski Dom,5.833333,2.0,226301964.0,0101000020e6100000a4c4aeeded622240710ace482973...,9.193122,48.899643,0.053982
233295401,place_of_worship,434.866,"POLYGON ((9.190878799999998 48.88974909971967,...",POINT (9.191010524676759 48.88970627502415),jesus_AG,3.213115,1.0,NaN,NaN,9.191011,48.889706,0.043487
241360980,place_of_worship,386.289,"POLYGON ((9.1869561 48.89293599971887, 9.18695...",POINT (9.187125169043505 48.8929043677569),Christliche Versammlungen,1.086957,1.0,NaN,NaN,9.187125,48.892904,0.038629
547490603,place_of_worship,214.686,"POLYGON ((9.176052800000001 48.88858619971995,...",POINT (9.176133871332219 48.88861634882115),Paul-Gerhardt-Gemeindehaus,1.000000,1.0,547318410.0,0101000020e61000006118b0e42a5a22409b65c9dfbd71...,9.176134,48.888616,0.021469


In [12]:
buildings_w_loads_deprecated.sort_values('capacity', ascending=False)

,category,capacity,area,number_households,x,y,geometry,raccordement_building
osm_id,,,,,,,,
208859161,industrial,6145.160700,37356.6,0.666667,9.16617,48.891829,"POLYGON ((9.165388699999999 48.89100459971936,...",POINT (9.166169849276244 48.89182932279353)
208885938,industrial,5289.793600,32156.8,1.489362,9.171253,48.893714,"POLYGON ((9.170302 48.8932507997188, 9.1706538...",POINT (9.171253250360374 48.89371427246718)
132279038,industrial,2602.274850,15819.3,0.0,9.167003,48.893402,"POLYGON ((9.166468 48.8940049997186, 9.1664683...",POINT (9.167003435859485 48.89340192480272)
132279034,industrial,2039.207800,12396.4,1.823529,9.16597,48.89363,"POLYGON ((9.165444600000001 48.89376189971868,...",POINT (9.16596954408331 48.89362979538151)
139755343,industrial,1955.230550,11885.9,0.787879,9.178967,48.888447,"POLYGON ((9.1784263 48.88814799972007, 9.17889...",POINT (9.178967405712543 48.88844672032332)
...,...,...,...,...,...,...,...,...
567262993,place_of_worship,0.053982,1079.64,5.833333,9.193122,48.899643,"POLYGON ((9.192986599999999 48.89974089971719,...",POINT (9.19312181385466 48.89964325600684)
233295401,place_of_worship,0.043487,434.866,3.213115,9.191011,48.889706,"POLYGON ((9.190878799999998 48.88974909971967,...",POINT (9.191010524676759 48.88970627502415)
241360980,place_of_worship,0.038629,386.289,1.086957,9.187125,48.892904,"POLYGON ((9.1869561 48.89293599971887, 9.18695...",POINT (9.187125169043505 48.8929043677569)


In [118]:
ids = buildings_w_loads_df.index
dupl_ids = list(set(buildings_w_loads_df[ids.isin(ids[ids.duplicated()])].index.tolist()))

if len(dupl_ids) > 0: # drop dupl if exists e.g. one amenity is located in multiple buildings.

    dupl_df = buildings_w_loads_df.loc[buildings_w_loads_df.index.isin(dupl_ids)]

    for ix, dupl_id in enumerate(dupl_ids):

        if ix < 1:

            dupl_dff = dupl_df[(dupl_df.index==dupl_id) & (dupl_df.area!=dupl_df.loc[dupl_df.index==dupl_id].area.max())]

        else:

            tempf_df = dupl_df[(dupl_df.index==dupl_id) & (dupl_df.area!=dupl_df.loc[dupl_df.index==dupl_id].area.max())]
            dupl_dff = pd.concat([dupl_dff, tempf_df])

    buildings_w_loads_df = buildings_w_loads_df[~buildings_w_loads_df.index.isin(dupl_ids)]

    buildings_w_loads_df = pd.concat([buildings_w_loads_df, dupl_dff])
